# Benchmark Results Analysis

This notebook analyzes benchmark results from `results/summaries.json` and generates summary statistics for the technical report.

In [2]:
import json
from pathlib import Path

import pandas as pd

# Load benchmark results (use absolute path for notebook reliability)
notebook_dir = Path("/workspace/experiments/benchmarks")
results_path = notebook_dir / "results/summaries.json"
with open(results_path) as f:
    data = json.load(f)

print(f"Loaded {len(data)} benchmark results")
print(f"Sample record keys: {list(data[0].keys())}")

Loaded 100 benchmark results
Sample record keys: ['algorithm', 'process', 'sample_size', 'expected_states', 'repetitions', 'runtime_mean', 'runtime_std', 'runtime_median', 'runtime_min', 'runtime_max', 'memory_mean', 'memory_std', 'memory_median', 'num_states_mode', 'correct_rate', 'success_rate']


In [3]:
# Convert to DataFrame for easier analysis
df = pd.DataFrame(data)
# Rename for consistency
df = df.rename(columns={"sample_size": "num_samples", "num_states_mode": "num_states"})
df.head()

algorithm      process  num_samples  expected_states  repetitions  \
0      cssr  golden_mean          100                2            5   
1      cssr  golden_mean         1000                2            5   
2      cssr  golden_mean        10000                2            5   
3      cssr  golden_mean       100000                2            5   
4      cssr  golden_mean      1000000                2            5   

   runtime_mean  runtime_std  runtime_median  runtime_min  runtime_max  \
0      0.002839     0.000149        0.002894     0.002590     0.002948   
1      0.013130     0.000249        0.013170     0.012868     0.013378   
2      0.133400     0.000657        0.132996     0.132854     0.134221   
3      1.345396     0.003871        1.344737     1.340610     1.351303   
4     13.453081     0.036132       13.446621    13.419231    13.507327   

   memory_mean  memory_std  memory_median  num_states  correct_rate  \
0      29157.6  432.231767        29286.0           2           1.0   
1      38557.0   12.649111        38557.0           2           1.0   
2     112648.2   16.589153       112645.0           2           1.0   
3     832723.4   15.388307       832725.0           2           1.0   
4    8032685.0    0.000000      8032685.0           2           1.0   

   success_rate  
0           1.0  
1           1.0  
2           1.0  
3           1.0  
4           1.0

## Overall Correctness by Algorithm

In [4]:
# Overall correctness by algorithm (using correct_rate from the aggregated data)
# Weight by number of test cases (all same here, but good practice)
overall = df.groupby("algorithm")["correct_rate"].mean() * 100
overall = overall.sort_values(ascending=False)
print("Overall Correctness by Algorithm")
print("=" * 40)
for alg, pct in overall.items():
    print(f"{alg:15s} {pct:5.1f}%")

Overall Correctness by Algorithm
spectral         85.0%
cssr             82.0%
nsd              73.0%
csm              39.0%
bsi              32.0%


## Correctness by Algorithm and Sample Size

In [5]:
# Correctness by algorithm and sample size
by_size = (
    df.pivot_table(index="algorithm", columns="num_samples", values="correct_rate", aggfunc="mean")
    * 100
)

print("Correctness (%) by Algorithm and Sample Size")
print(by_size.round(0).to_string())

Correctness (%) by Algorithm and Sample Size
num_samples  100      1000     10000    100000   1000000
algorithm                                               
bsi             60.0     25.0     25.0     25.0     25.0
csm             15.0     30.0     50.0     50.0     50.0
cssr            90.0     95.0     75.0     75.0     75.0
nsd             65.0     75.0     75.0     75.0     75.0
spectral        35.0     90.0    100.0    100.0    100.0


## Correctness for N ≥ 1,000

In [6]:
# Correctness for N >= 1000 only (what we report in the summary table)
large_n = df[df["num_samples"] >= 1000]
large_n_correct = large_n.groupby("algorithm")["correct_rate"].mean() * 100
large_n_correct = large_n_correct.sort_values(ascending=False)

print("Correctness at N >= 1,000")
print("=" * 40)
for alg, pct in large_n_correct.items():
    print(f"{alg:15s} {pct:5.1f}%")

Correctness at N >= 1,000
spectral         97.5%
cssr             80.0%
nsd              75.0%
csm              45.0%
bsi              25.0%


## Correctness by Algorithm and Process

In [7]:
# Correctness by algorithm and process (for N >= 1000)
by_process = (
    large_n.pivot_table(index="process", columns="algorithm", values="correct_rate", aggfunc="mean")
    * 100
)

print("Correctness (%) by Process and Algorithm (N >= 1,000)")
print(by_process.round(0).to_string())

Correctness (%) by Process and Algorithm (N >= 1,000)
algorithm       bsi    csm   cssr    nsd  spectral
process                                           
biased_coin   100.0   80.0   95.0  100.0     100.0
even_process    0.0    0.0   25.0  100.0     100.0
golden_mean     0.0  100.0  100.0  100.0      90.0
periodic        0.0    0.0  100.0    0.0     100.0


## State Counts at N=100,000

In [8]:
# State counts at N=100,000
n100k = df[df["num_samples"] == 100000]
state_counts = n100k.pivot_table(
    index="process",
    columns="algorithm",
    values="num_states",
    aggfunc="first",  # Mode is already computed
)

# Also show expected states
state_counts["expected"] = n100k.groupby("process")["expected_states"].first()

print("State Count at N=100,000 (mode across repetitions)")
print(state_counts.to_string())

State Count at N=100,000 (mode across repetitions)
algorithm     bsi  csm  cssr  nsd  spectral  expected
process                                              
biased_coin     1    1     1    1         1         1
even_process    4    3     4    2         2         2
golden_mean     3    2     2    2         2         2
periodic        4    2     3    2         3         3


## Runtime Statistics

In [ ]:
# Runtime at N=1,000,000
n1m = df[df["num_samples"] == 1000000]
runtime = n1m.groupby("algorithm")[["runtime_mean", "runtime_std"]].first()
runtime = runtime.sort_values("runtime_mean")

print("Runtime at N=1,000,000 (seconds)")
print("=" * 50)
for alg, row in runtime.iterrows():
    print(f"{alg:15s} {row['runtime_mean']:8.2f}s (±{row['runtime_std']:.2f})")

Runtime at N=1,000,000 (seconds)
csm                 1.17s (±0.00)
nsd                 4.02s (±0.01)
cssr               13.45s (±0.04)
bsi                21.04s (±0.05)
spectral           39.67s (±0.06)


## LaTeX Table for Technical Report

In [11]:
# Generate LaTeX table for correctness by sample size
print("% LaTeX table: Correctness by Algorithm and Sample Size")
print("\\begin{tabular}{l" + "r" * len(by_size.columns) + "}")
print("\\toprule")
cols = [f"$N={n:,}$".replace(",", "{,}") for n in by_size.columns]
header = "\\textbf{Algorithm} & " + " & ".join(cols) + " \\\\"
print(header)
print("\\midrule")
for alg, row in by_size.iterrows():
    values = " & ".join([f"{v:.0f}\\%" for v in row.values])
    print(f"{alg} & {values} \\\\")
print("\\bottomrule")
print("\\end{tabular}")

% LaTeX table: Correctness by Algorithm and Sample Size
\begin{tabular}{lrrrrr}
\toprule
\textbf{Algorithm} & $N=100$ & $N=1{,}000$ & $N=10{,}000$ & $N=100{,}000$ & $N=1{,}000{,}000$ \\
\midrule
bsi & 60\% & 25\% & 25\% & 25\% & 25\% \\
csm & 15\% & 30\% & 50\% & 50\% & 50\% \\
cssr & 90\% & 95\% & 75\% & 75\% & 75\% \\
nsd & 65\% & 75\% & 75\% & 75\% & 75\% \\
spectral & 35\% & 90\% & 100\% & 100\% & 100\% \\
\bottomrule
\end{tabular}


## Failure Mode Analysis

In [ ]:
# Failure mode analysis: which processes fail for each algorithm?
failures = df[df["correct_rate"] < 1.0].copy()

print("Failure Modes by Algorithm and Process")
print("=" * 60)
for alg in sorted(failures["algorithm"].unique()):
    alg_failures = failures[failures["algorithm"] == alg]
    print(f"\n{alg.upper()}:")
    for _, row in alg_failures.iterrows():
        if row["correct_rate"] < 1.0:
            print(
                f"  {row['process']} @ N={row['num_samples']:,}: "
                f"found {row['num_states']} states (expected {row['expected_states']}), "
                f"correct {row['correct_rate'] * 100:.0f}%"
            )

Failure Modes by Algorithm and Process

BSI:
  golden_mean @ N=100: found 2 states (expected 2), correct 60%
  golden_mean @ N=1,000: found 3 states (expected 2), correct 0%
  golden_mean @ N=10,000: found 3 states (expected 2), correct 0%
  golden_mean @ N=100,000: found 3 states (expected 2), correct 0%
  golden_mean @ N=1,000,000: found 3 states (expected 2), correct 0%
  even_process @ N=100: found 3 states (expected 2), correct 40%
  even_process @ N=1,000: found 4 states (expected 2), correct 0%
  even_process @ N=10,000: found 4 states (expected 2), correct 0%
  even_process @ N=100,000: found 4 states (expected 2), correct 0%
  even_process @ N=1,000,000: found 4 states (expected 2), correct 0%
  biased_coin @ N=100: found 2 states (expected 1), correct 40%
  periodic @ N=1,000: found 4 states (expected 3), correct 0%
  periodic @ N=10,000: found 4 states (expected 3), correct 0%
  periodic @ N=100,000: found 4 states (expected 3), correct 0%
  periodic @ N=1,000,000: found 4 s

## Summary for Technical Report

In [ ]:
# Summary of key metrics for technical report
print("=" * 60)
print("SUMMARY FOR TECHNICAL REPORT")
print("=" * 60)

print("\n1. Overall Correctness (all sample sizes):")
for alg, pct in overall.items():
    print(f"   {alg:12s}: {pct:.0f}%")

print("\n2. Correctness at N >= 1,000:")
for alg, pct in large_n_correct.items():
    print(f"   {alg:12s}: {pct:.0f}%")

print("\n3. Runtime at N=1,000,000:")
for alg, row in runtime.iterrows():
    print(f"   {alg:12s}: {row['runtime_mean']:.1f}s")

print("\n4. Key observations:")
print("   - Spectral achieves 100% correctness at N >= 10,000")
print("   - CSSR degrades at high N (Even Process over-splitting)")
print("   - NSD fails on deterministic (Periodic) processes")
print("   - CSM fails on Even Process and Periodic")
print("   - BSI fails on most structured processes")

SUMMARY FOR TECHNICAL REPORT

1. Overall Correctness (all sample sizes):
   spectral    : 85%
   cssr        : 82%
   nsd         : 73%
   csm         : 39%
   bsi         : 32%

2. Correctness at N >= 1,000:
   spectral    : 98%
   cssr        : 80%
   nsd         : 75%
   csm         : 45%
   bsi         : 25%

3. Runtime at N=1,000,000:
   csm         : 1.2s
   nsd         : 4.0s
   cssr        : 13.5s
   bsi         : 21.0s
   spectral    : 39.7s

4. Key observations:
   - Spectral achieves 100% correctness at N >= 10,000
   - CSSR degrades at high N (Even Process over-splitting)
   - NSD fails on deterministic (Periodic) processes
   - CSM fails on Even Process and Periodic
   - BSI fails on most structured processes


## LaTeX Output for Technical Report

Run this cell to generate all LaTeX tables. Copy/paste into `technical-report.tex` or save to a file that can be `\input{}`.

In [ ]:
from pathlib import Path

# Output directory for generated LaTeX
output_dir = Path("/workspace/.project/research/papers/technical-report/tex/generated")
output_dir.mkdir(exist_ok=True)


def write_latex(filename: str, content: str):
    """Write LaTeX content to file and print it."""
    path = output_dir / filename
    path.write_text(content)
    print(f"% Written to: {path}")
    print(content)
    print()


# ============================================================
# TABLE 1: State Counts at N=100,000 (tab:state-counts)
# ============================================================
n100k = df[df["num_samples"] == 100000]

latex_state_counts = r"""\begin{table}[h]
\centering
\caption{Inferred state counts at $N = 100{,}000$ (from benchmark data)}
\label{tab:state-counts}
\begin{tabular}{lcccccc}
\toprule
Process & True & CSSR & Spectral & NSD & CSM & BSI \\
\midrule
"""

process_names = {
    "biased_coin": "BiasedCoin",
    "golden_mean": "GoldenMean",
    "even_process": "EvenProcess",
    "periodic": "Periodic",
}
process_order = ["biased_coin", "golden_mean", "even_process", "periodic"]

for proc in process_order:
    proc_data = n100k[n100k["process"] == proc]
    expected = proc_data["expected_states"].iloc[0]

    states = {}
    for _, row in proc_data.iterrows():
        states[row["algorithm"]] = int(row["num_states"])

    latex_state_counts += f"{process_names[proc]} & {expected} & {states.get('cssr', '?')} & {states.get('spectral', '?')} & {states.get('nsd', '?')} & {states.get('csm', '?')} & {states.get('bsi', '?')} \\\\\n"

latex_state_counts += r"""\bottomrule
\end{tabular}
\end{table}"""

write_latex("tab-state-counts.tex", latex_state_counts)

# ============================================================
# TABLE 2: Correctness Summary (tab:correctness)
# ============================================================
# For N >= 1000 (4 sample sizes: 1K, 10K, 100K, 1M)
large_n = df[df["num_samples"] >= 1000]

latex_correctness = r"""\begin{table}[htbp]
\centering
\begin{tabular}{lccccc}
\toprule
\textbf{Process} & \textbf{CSSR} & \textbf{Spectral} & \textbf{NSD} & \textbf{CSM} & \textbf{BSI} \\
\midrule
"""

expected_states = {"biased_coin": 1, "golden_mean": 2, "even_process": 2, "periodic": 3}
totals = dict.fromkeys(["cssr", "spectral", "nsd", "csm", "bsi"], 0)
max_per_process = 4  # 4 sample sizes >= 1000

for proc in process_order:
    proc_data = large_n[large_n["process"] == proc]
    expected = expected_states[proc]

    scores = {}
    for alg in ["cssr", "spectral", "nsd", "csm", "bsi"]:
        alg_data = proc_data[proc_data["algorithm"] == alg]
        # correct_rate is per sample size, count how many are 100%
        correct_count = int((alg_data["correct_rate"] == 1.0).sum())
        scores[alg] = correct_count
        totals[alg] += correct_count

    proc_label = f"{process_names[proc]} ({expected})"
    latex_correctness += f"{proc_label} & {scores['cssr']}/{max_per_process} & {scores['spectral']}/{max_per_process} & {scores['nsd']}/{max_per_process} & {scores['csm']}/{max_per_process} & {scores['bsi']}/{max_per_process} \\\\\n"

# Totals
max_total = max_per_process * len(process_order)
latex_correctness += r"\midrule" + "\n"
latex_correctness += f"\\textbf{{Overall}} & {totals['cssr']}/{max_total} & \\textbf{{{totals['spectral']}/{max_total}}} & {totals['nsd']}/{max_total} & {totals['csm']}/{max_total} & {totals['bsi']}/{max_total} \\\\\n"
latex_correctness += f" & ({100 * totals['cssr'] / max_total:.0f}\\%) & \\textbf{{({100 * totals['spectral'] / max_total:.0f}\\%)}} & ({100 * totals['nsd'] / max_total:.0f}\\%) & ({100 * totals['csm'] / max_total:.0f}\\%) & ({100 * totals['bsi'] / max_total:.0f}\\%) \\\\\n"

latex_correctness += r"""\bottomrule
\end{tabular}
\caption{Correctness by algorithm and process for $N \geq 1{,}000$. Each cell shows the number of sample sizes (out of 4: 1K, 10K, 100K, 1M) where the algorithm found the correct number of states. Numbers in parentheses indicate expected state count.}
\label{tab:correctness}
\end{table}"""

write_latex("tab-correctness.tex", latex_correctness)

# ============================================================
# TABLE 3: Performance Summary (tab:perf-summary)
# ============================================================
n1m = df[df["num_samples"] == 1000000]

# Get runtime means
runtime_means = {}
for alg in ["cssr", "spectral", "csm", "bsi", "nsd"]:
    alg_data = n1m[n1m["algorithm"] == alg]
    runtime_means[alg] = alg_data["runtime_mean"].mean()

# Get correctness at N >= 1000
correctness_large = {}
for alg in ["cssr", "spectral", "csm", "bsi", "nsd"]:
    alg_data = large_n[large_n["algorithm"] == alg]
    correctness_large[alg] = alg_data["correct_rate"].mean() * 100

cssr_time = runtime_means["cssr"]

latex_perf = r"""\begin{table}[htbp]
\centering
\begin{tabular}{lccccc}
\toprule
\textbf{Metric} & \textbf{CSSR} & \textbf{Spectral} & \textbf{CSM} & \textbf{BSI} & \textbf{NSD} \\
\midrule
"""

# Time row
latex_perf += f"Time @ $N=1$M & {runtime_means['cssr']:.1f} s & {runtime_means['spectral']:.1f} s & {runtime_means['csm']:.1f} s & {runtime_means['bsi']:.1f} s & {runtime_means['nsd']:.1f} s \\\\\n"

# Speedup row
speedups = {alg: cssr_time / runtime_means[alg] for alg in runtime_means}
latex_perf += f"Speedup vs CSSR & 1$\\times$ & {speedups['spectral']:.1f}$\\times$ & {speedups['csm']:.0f}$\\times$ & {speedups['bsi']:.1f}$\\times$ & {speedups['nsd']:.1f}$\\times$ \\\\\n"

# Correctness row - bold the best
best_alg = max(correctness_large, key=correctness_large.get)
correctness_strs = {}
for alg in ["cssr", "spectral", "csm", "bsi", "nsd"]:
    pct = f"{correctness_large[alg]:.0f}\\%"
    if alg == best_alg:
        correctness_strs[alg] = f"\\textbf{{{pct}}}"
    else:
        correctness_strs[alg] = pct

latex_perf += f"Correctness ($N \\geq 1$K) & {correctness_strs['cssr']} & {correctness_strs['spectral']} & {correctness_strs['csm']} & {correctness_strs['bsi']} & {correctness_strs['nsd']} \\\\\n"

latex_perf += r"""\bottomrule
\end{tabular}
\caption{Summary of performance characteristics across all algorithms.}
\label{tab:perf-summary}
\end{table}"""

write_latex("tab-perf-summary.tex", latex_perf)

# ============================================================
# TABLE 4: Detailed Correctness by Sample Size (tab:correctness-detail)
# ============================================================
sample_sizes = [100, 1000, 10000, 100000, 1000000]


def get_result_symbol(proc, alg, n):
    """Get checkmark or x with state count."""
    row = df[(df["process"] == proc) & (df["algorithm"] == alg) & (df["num_samples"] == n)]
    if len(row) == 0:
        return "?"
    expected = expected_states[proc]
    actual = int(row["num_states"].iloc[0])
    if actual == expected:
        return r"\checkmark"
    else:
        return f"$\\times${actual}"


latex_detail = r"""\begin{table}[htbp]
\centering
\footnotesize
\begin{tabular}{l|ccccc|ccccc}
\toprule
& \multicolumn{5}{c|}{\textbf{Golden Mean (2 states)}} & \multicolumn{5}{c}{\textbf{Even Process (2 states)}} \\
\textbf{N} & 100 & 1K & 10K & 100K & 1M & 100 & 1K & 10K & 100K & 1M \\
\midrule
"""

for alg in ["cssr", "spectral", "nsd", "csm", "bsi"]:
    alg_upper = alg.upper()
    gm_results = [get_result_symbol("golden_mean", alg, n) for n in sample_sizes]
    ep_results = [get_result_symbol("even_process", alg, n) for n in sample_sizes]
    latex_detail += f"{alg_upper} & {' & '.join(gm_results)} & {' & '.join(ep_results)} \\\\\n"

latex_detail += r"""\midrule
& \multicolumn{5}{c|}{\textbf{Biased Coin (1 state)}} & \multicolumn{5}{c}{\textbf{Periodic (3 states)}} \\
\textbf{N} & 100 & 1K & 10K & 100K & 1M & 100 & 1K & 10K & 100K & 1M \\
\midrule
"""

for alg in ["cssr", "spectral", "nsd", "csm", "bsi"]:
    alg_upper = alg.upper()
    bc_results = [get_result_symbol("biased_coin", alg, n) for n in sample_sizes]
    per_results = [get_result_symbol("periodic", alg, n) for n in sample_sizes]
    latex_detail += f"{alg_upper} & {' & '.join(bc_results)} & {' & '.join(per_results)} \\\\\n"

latex_detail += r"""\bottomrule
\end{tabular}
\caption{Detailed correctness by sample size. \checkmark = correct state count; $\times n$ = inferred $n$ states (wrong).}
\label{tab:correctness-detail}
\end{table}"""

write_latex("tab-correctness-detail.tex", latex_detail)

print("=" * 70)
print("All tables written to:", output_dir)
print("To use in technical-report.tex, add: \\input{generated/tab-XXX.tex}")
print("=" * 70)

% Written to: /workspace/.project/research/papers/technical-report/tex/generated/tab-state-counts.tex
\begin{table}[h]
\centering
\caption{Inferred state counts at $N = 100{,}000$ (from benchmark data)}
\label{tab:state-counts}
\begin{tabular}{lcccccc}
\toprule
Process & True & CSSR & Spectral & NSD & CSM & BSI \\
\midrule
BiasedCoin & 1 & 1 & 1 & 1 & 1 & 1 \\
GoldenMean & 2 & 2 & 2 & 2 & 2 & 3 \\
EvenProcess & 2 & 4 & 2 & 2 & 3 & 4 \\
Periodic & 3 & 3 & 3 & 2 & 2 & 4 \\
\bottomrule
\end{tabular}
\end{table}

% Written to: /workspace/.project/research/papers/technical-report/tex/generated/tab-correctness.tex
\begin{table}[htbp]
\centering
\begin{tabular}{lccccc}
\toprule
\textbf{Process} & \textbf{CSSR} & \textbf{Spectral} & \textbf{NSD} & \textbf{CSM} & \textbf{BSI} \\
\midrule
BiasedCoin (1) & 3/4 & 4/4 & 4/4 & 3/4 & 4/4 \\
GoldenMean (2) & 4/4 & 3/4 & 4/4 & 4/4 & 0/4 \\
EvenProcess (2) & 1/4 & 4/4 & 4/4 & 0/4 & 0/4 \\
Periodic (3) & 4/4 & 4/4 & 0/4 & 0/4 & 0/4 \\
\midrule
\textbf{Ov